In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%set_env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [3]:
import sys
import collections
import cv2
import os
import math
import random
import pickle
import copy
import numpy as np
import pandas as pd
sys.path.append("C:\\Users\\user\\Documents\\Workspace\\MeronymNet-PyTorch\\src")

In [13]:
import torch
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.utils import to_dense_adj
from torch_geometric.data import DataLoader
import torch.utils.data as data_utils
from torch.utils.tensorboard import SummaryWriter

In [27]:
from losses import BoxVAE_losses as loss
from evaluation import metrics
from utils import plot_utils
from utils import data_utils as data_loading
from components.DenseAutoencoder import DenseAutoencoder
from components.DenseAutoencoder import Decoder
from components.TwoStageAutoEncoder import TwoStageAutoEncoder

In [6]:
import matplotlib.pyplot as plt

In [7]:
# from mask_generation import masked_sketch

In [9]:
batch_size = 128
latent_dims = 64
num_nodes = 16
bbx_size = 4
num_classes = 7
label_shape = 1
nb_epochs = 1000
klw = loss.frange_cycle_linear(nb_epochs)
learning_rate = 0.000056
hidden1 = 32
hidden2 = 16
hidden3 = 128
dense_hidden1=16
dense_hidden2=32
adaptive_margin = True
fine_tune_box = False
output_log = False
area_encoding = False
run_prefix = "two_stage_gated_adjacency"
variational=False

In [10]:
batch_train_loader, batch_val_loader = data_loading.load_data(obj_data_postfix = '_obj_boundary',
                                                              part_data_post_fix = '_scaled',
                                                              file_postfix = '_combined',
                                                              seed=345,
                                                              batch_size=batch_size)

C:\Users\user\miniconda3\lib\site-packages\torch_geometric\deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [17]:
import gc
gc.collect()

92

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [28]:
reconstruction_loss_arr = []
kl_loss_arr = []
bbox_loss_arr = []
adj_loss_arr = []
node_loss_arr = []

reconstruction_loss_val_arr = []
kl_loss_val_arr = []
bbox_loss_val_arr = []
adj_loss_val_arr = []
node_loss_val_arr = []

vae = TwoStageAutoEncoder(latent_dims,
                          num_nodes,
                          bbx_size,
                          num_classes,
                          label_shape,
                          hidden1,
                          hidden2,
                          hidden3,
                          dense_hidden1,
                          dense_hidden2,
                          adaptive_margin,
                          output_log,
                          area_encoding
                        )
vae.to(device)
optimizer = optim.Adam(vae.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[200,400], gamma=0.75)
model_path = ('D:/meronym_data/model/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
summary_path = ('D:/meronym_data/runs/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
if not os.path.exists(model_path):
    os.makedirs(model_path)
writer = SummaryWriter(summary_path)
icoef = 0
start = 0
for epoch in range(start, nb_epochs):
    
    batch_loss = torch.tensor([0.0])
    batch_kl_loss = torch.tensor([0.0])
    batch_bbox_loss = torch.tensor([0.0])
    batch_obj_bbox_loss = torch.tensor([0.0])
    batch_node_loss = torch.tensor([0.0])
    batch_pair_loss = torch.tensor([0.0])
    IOU_weight_delta = torch.tensor([(1+epoch)/nb_epochs])
    images = []
    
    vae.train()
    i=0
    for train_data in batch_train_loader:
        
        node_data_true = train_data.x
        label_true = node_data_true[:,:1]
        y_true = train_data.y
        class_true = y_true[:, :num_classes]
        X_obj_true = y_true[:, num_classes:]
        adj_true = train_data.edge_index
        batch = train_data.batch
        
        class_true  = torch.flatten(class_true)
        

        for param in vae.parameters():
            param.grad=None
        
        output = vae(adj_true, node_data_true, X_obj_true, label_true , class_true, False)
        node_data_pred, X_obj_pred, label_pred, z_mean, z_logvar, margin = output
        
        adj_true=torch.flatten(to_dense_adj(adj_true, batch), start_dim=1)
        obj_bbox_loss = loss.obj_bbox_loss(pred_box=X_obj_pred, true_box=X_obj_true, has_mse=False)
        kl_loss = loss.kl_loss(z_mean, z_logvar)
        bbox_loss = loss.weighted_bbox_loss(pred_box=node_data_pred, true_box=node_data_true[:,1:], weight=IOU_weight_delta, margin=margin)
        node_loss = loss.node_loss(label_pred,label_true)
        pairwise_loss = loss.intersection_loss_adjacency_gated(pred_box=node_data_pred, true_box=node_data_true[:,1:],
                                                               adj_true=adj_true, log_output=False, area_encoding=False,
                                                               binary=False)
        
        kl_weight = klw[icoef]
        
        if kl_weight>0:
            reconstruction_loss = kl_loss*kl_weight + (obj_bbox_loss*10 + bbox_loss + node_loss + pairwise_loss*num_nodes)*num_nodes
        else:
            reconstruction_loss = (obj_bbox_loss*10 + bbox_loss + node_loss + pairwise_loss*num_nodes)*num_nodes
            
        reconstruction_loss.backward()
        
        optimizer.step()
        
        i+=1
      
        batch_loss += reconstruction_loss
        batch_kl_loss += kl_loss
        batch_bbox_loss += bbox_loss
        batch_obj_bbox_loss += obj_bbox_loss
        batch_node_loss += node_loss
        batch_pair_loss += pairwise_loss
    
        if i%200==0:
            print(i)
            global_step = epoch*len(batch_train_loader)+i
            
            writer.add_scalar("Loss/train/reconstruction_loss", batch_loss.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/kl_loss", batch_kl_loss.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/bbox_loss", batch_bbox_loss.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/obj_bbox_loss", batch_obj_bbox_loss.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/node_loss", batch_node_loss.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/pair_loss", batch_pair_loss.item()/(i+1), global_step)
            
            
#     scheduler.step()
    global_step = epoch*len(batch_train_loader)+i
    image_shape = [num_nodes, bbx_size]

    image = plot_utils.plot_bbx(np.reshape((node_data_true[:num_nodes,1:5]*label_true[:num_nodes]).detach().to("cpu").numpy(),
                                image_shape)).astype(float)/255
    writer.add_image('train/images/input', image, global_step, dataformats='HWC')
    image = plot_utils.plot_bbx((node_data_pred[0]*label_true[:num_nodes]).detach().to("cpu").numpy()).astype(float)/255
    writer.add_image('train/images/generated', image, global_step, dataformats='HWC')
    
    reconstruction_loss_arr.append(batch_loss.detach().item()/(i+1))
    kl_loss_arr.append(batch_kl_loss.detach().item()/(i+1))
    bbox_loss_arr.append(batch_bbox_loss.detach().item()/(i+1))
    node_loss_arr.append(batch_node_loss.detach().item()/(i+1))
    
    print('[%d, %5d] loss: %.3f' %
              (epoch + 1, i + 1, batch_loss/(i+1) ))
    
    
    batch_loss = torch.tensor([0.0])
    batch_kl_loss = torch.tensor([0.0])
    batch_bbox_loss = torch.tensor([0.0])
    batch_obj_bbox_loss = torch.tensor([0.0])
    batch_node_loss = torch.tensor([0.0])
    batch_pair_loss = torch.tensor([0.0])
    images = []
    vae.eval()
    for i, val_data in enumerate(batch_val_loader, 0):
        
        node_data_true = val_data.x
        label_true = node_data_true[:,:1]
        y_true = val_data.y
        class_true = torch.flatten(y_true[:, :num_classes])
        X_obj_true = y_true[:, num_classes:]
        adj_true = val_data.edge_index
        batch = val_data.batch
        
        class_true  = torch.flatten(class_true)
        
        output = vae(adj_true, node_data_true, X_obj_true, label_true , class_true, False)
        node_data_pred, X_obj_pred, label_pred, z_mean, z_logvar, margin = output
        obj_bbox_loss = loss.obj_bbox_loss(pred_box=X_obj_pred, true_box=X_obj_true, has_mse=False)
        kl_loss = loss.kl_loss(z_mean, z_logvar)
        bbox_loss = loss.weighted_bbox_loss(pred_box=node_data_pred, true_box=node_data_true[:,1:], weight=IOU_weight_delta, margin=margin)
        node_loss = loss.node_loss(label_pred,label_true)
        pairwise_loss = loss.intersection_loss(pred_box=node_data_pred, true_box=node_data_true[:,1:],
                                               log_output=False, area_encoding=False, binary=False)
        
        if kl_weight>0:
            reconstruction_loss = kl_loss*kl_weight + (obj_bbox_loss*10 + bbox_loss + node_loss)*num_nodes
        else:
            reconstruction_loss = (obj_bbox_loss*10 + bbox_loss + node_loss + pairwise_loss)*num_nodes
            
        batch_loss += reconstruction_loss
        batch_kl_loss += kl_loss
        batch_bbox_loss += bbox_loss
        batch_obj_bbox_loss += obj_bbox_loss
        batch_node_loss += node_loss
        batch_pair_loss += pairwise_loss
    
    image = plot_utils.plot_bbx(np.reshape((node_data_true[:num_nodes,1:5]*label_true[:num_nodes]).detach().to("cpu").numpy(),
                                image_shape)).astype(float)/255
    writer.add_image('val/images/input', image, global_step, dataformats='HWC')
    image = plot_utils.plot_bbx((node_data_pred[0]*label_true[:num_nodes]).detach().to("cpu").numpy()).astype(float)/255
    writer.add_image('val/images/generated', image, global_step, dataformats='HWC')

    reconstruction_loss_val_arr.append(batch_loss.detach().item()/(i+1))
    kl_loss_val_arr.append(batch_kl_loss.detach().item()/(i+1))
    bbox_loss_val_arr.append(batch_bbox_loss.detach().item()/(i+1))
    node_loss_val_arr.append(batch_node_loss.detach().item()/(i+1))
    
    writer.add_scalar("Loss/val/reconstruction_loss", batch_loss.detach()/(i+1), global_step)
    writer.add_scalar("Loss/val/kl_loss", batch_kl_loss.item()/(i+1), global_step)
    writer.add_scalar("Loss/val/bbox_loss", batch_bbox_loss.item()/(i+1), global_step)
    writer.add_scalar("Loss/val/obj_bbox_loss", batch_obj_bbox_loss.item()/(i+1), global_step)
    writer.add_scalar("Loss/val/node_loss", batch_node_loss.item()/(i+1), global_step)
    writer.add_scalar("Loss/val/pair_loss", batch_pair_loss.item()/(i+1), global_step)
       
    if epoch%50 == 0:
        torch.save(vae.state_dict(), model_path + '/model_weights.pth')
        
#     if kl_loss_arr[-1]>0.5 and abs(bbox_loss_arr[-1] - bbox_loss_val_arr[-1]) < 0.02 and bbox_loss_arr[-1]<1.3 and epoch>60:
#         icoef = icoef + 1  

torch.save(vae.state_dict(),model_path + '/model_weights.pth')

for i in range(min(100,int(len(node_data_true)/num_nodes))):    
    image = plot_utils.plot_bbx(np.reshape((node_data_true[num_nodes*i:num_nodes*(i+1),1:5]).detach().to("cpu").numpy(),
                                    image_shape)).astype(float)/255
    writer.add_image('result/images/'+str(i)+'-input', image, global_step, dataformats='HWC')
    image = plot_utils.plot_bbx((node_data_pred[i]*label_true[num_nodes*i:num_nodes*(i+1)]).detach().to("cpu").numpy()).astype(float)/255
    writer.add_image('result/images/'+str(i)+'-generated', image, global_step, dataformats='HWC')
    
writer.flush()
writer.close()
print('Finished Training')

200
[1,   399] loss: 224.746
200
[2,   399] loss: 154.029
200
[3,   399] loss: 137.208
200
[4,   399] loss: 133.876
200
[5,   399] loss: 131.081
200
[6,   399] loss: 127.418
200
[7,   399] loss: 123.729
200
[8,   399] loss: 121.623
200
[9,   399] loss: 120.523
200
[10,   399] loss: 116.699
200
[11,   399] loss: 102.845
200
[12,   399] loss: 87.042
200
[13,   399] loss: 79.746
200
[14,   399] loss: 76.060
200
[15,   399] loss: 73.743
200
[16,   399] loss: 72.057
200
[17,   399] loss: 70.511
200
[18,   399] loss: 68.846
200
[19,   399] loss: 67.067
200
[20,   399] loss: 65.146
200
[21,   399] loss: 63.133
200
[22,   399] loss: 61.005
200
[23,   399] loss: 58.699
200
[24,   399] loss: 56.509
200
[25,   399] loss: 54.656
200
[26,   399] loss: 53.225
200
[27,   399] loss: 52.029
200
[28,   399] loss: 51.038
200
[29,   399] loss: 50.229
200
[30,   399] loss: 49.579
200
[31,   399] loss: 49.011
200
[32,   399] loss: 48.483
200
[33,   399] loss: 47.985
200
[34,   399] loss: 47.500
200
[35,   3

200
[278,   399] loss: 15.474
200
[279,   399] loss: 15.444
200
[280,   399] loss: 15.409
200
[281,   399] loss: 15.380
200
[282,   399] loss: 15.348
200
[283,   399] loss: 15.317
200
[284,   399] loss: 15.300
200
[285,   399] loss: 15.264
200
[286,   399] loss: 15.235
200
[287,   399] loss: 15.201
200
[288,   399] loss: 15.186
200
[289,   399] loss: 15.149
200
[290,   399] loss: 15.113
200
[291,   399] loss: 15.087
200
[292,   399] loss: 15.051
200
[293,   399] loss: 15.024
200
[294,   399] loss: 15.011
200
[295,   399] loss: 14.967
200
[296,   399] loss: 14.946
200
[297,   399] loss: 14.918
200
[298,   399] loss: 14.894
200
[299,   399] loss: 14.846
200
[300,   399] loss: 14.823
200
[301,   399] loss: 14.791
200
[302,   399] loss: 14.771
200
[303,   399] loss: 14.746
200
[304,   399] loss: 14.705
200
[305,   399] loss: 14.680
200
[306,   399] loss: 14.669
200
[307,   399] loss: 14.629
200
[308,   399] loss: 14.602
200
[309,   399] loss: 14.575
200
[310,   399] loss: 14.542
200
[311, 

[551,   399] loss: 10.156
200
[552,   399] loss: 10.149
200
[553,   399] loss: 10.147
200
[554,   399] loss: 10.123
200
[555,   399] loss: 10.107
200
[556,   399] loss: 10.111
200
[557,   399] loss: 10.101
200
[558,   399] loss: 10.078
200
[559,   399] loss: 10.054
200
[560,   399] loss: 10.055
200
[561,   399] loss: 10.047
200
[562,   399] loss: 10.040
200
[563,   399] loss: 10.012
200
[564,   399] loss: 10.014
200
[565,   399] loss: 9.988
200
[566,   399] loss: 9.997
200
[567,   399] loss: 9.982
200
[568,   399] loss: 9.969
200
[569,   399] loss: 9.949
200
[570,   399] loss: 9.932
200
[571,   399] loss: 9.932
200
[572,   399] loss: 9.907
200
[573,   399] loss: 9.920
200
[574,   399] loss: 9.916
200
[575,   399] loss: 9.888
200
[576,   399] loss: 9.874
200
[577,   399] loss: 9.875
200
[578,   399] loss: 9.848
200
[579,   399] loss: 9.851
200
[580,   399] loss: 9.835
200
[581,   399] loss: 9.818
200
[582,   399] loss: 9.835
200
[583,   399] loss: 9.800
200
[584,   399] loss: 9.799
200


KeyboardInterrupt: 

In [29]:
#testing loop
model_path = ('D:/meronym_data/model/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
summary_path = ('D:/meronym_data/runs/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')


write_tensorboard = False
if write_tensorboard:
    writer = SummaryWriter(summary_path)

vae = TwoStageAutoEncoder(latent_dims,
                          num_nodes,
                          bbx_size,
                          num_classes,
                          label_shape,
                          hidden1,
                          hidden2,
                          hidden3,
                          dense_hidden1,
                          dense_hidden2,
                          adaptive_margin,
                          output_log,
                          area_encoding
                        )

vae.load_state_dict(torch.load(model_path+ '/model_weights.pth'))

# decoder = vae.decoder
image_shape = [num_nodes, bbx_size]
global_step = 250000
class_dict = {0:'cow', 1:'sheep', 2:'bird', 3:'person', 4:'cat', 5:'dog', 6:'horse'}
res_dfs = []
for i, val_data in enumerate(batch_val_loader, 0):
    
    val_data.cuda()
    node_data_true = val_data.x
    label_true = node_data_true[:,:1]
    y_true = val_data.y
    class_true = y_true[:, :num_classes]
    X_obj_true = y_true[:, num_classes:]
    adj_true = val_data.edge_index
    class_true  = torch.flatten(class_true)
    
    output = vae(adj_true, node_data_true, X_obj_true, label_true , class_true)
    node_data_pred_test, X_obj_pred_test, label_pred, z_mean, z_logvar, margin = output
    
    res_dfs.append(metrics.get_metrics(node_data_true, X_obj_true, node_data_pred_test,
                               X_obj_pred_test,
                               label_true,
                               class_true,
                               num_nodes,
                               num_classes))
    
    if write_tensorboard:
        
        for j in range(int(len(node_data_true)/num_nodes)):
            
            image = plot_utils.plot_bbx(node_data_true[j].detach().to("cpu").numpy().astype(float))/255
            pred_image = plot_utils.plot_bbx(node_data_pred_test[j].detach().to("cpu").numpy()).astype(float)/255

            writer.add_image('test_result/images/'+str(j)+'-input/', image, global_step, dataformats='HWC')  
            writer.add_image('test_result/images/'+str(j)+'-generated/', pred_image, global_step, dataformats='HWC')
            

result = pd.concat(res_dfs)
result['obj_class'] = np.where(result['obj_class'].isna(), 0, result['obj_class'])
result['obj_class'] = result['obj_class'].astype('int')
result['obj_class'].replace(class_dict, inplace=True)
result.where(result['part_labels']!=0, np.NaN, inplace=True)
result['part_labels'] = np.where(result['part_labels'].isna(), 0, result['part_labels'])
result['part_labels'] = result['part_labels'].astype('int')
result['id'] = np.repeat(np.array(list(range(int(len(result)/num_nodes)))), 16)

if write_tensorboard:
    writer.flush()
    writer.close()

bird_labels = {'head':1 , 'torso':2, 'neck':3, 'lwing':4, 'rwing':5, 'lleg':6, 'lfoot':7, 'rleg':8, 'rfoot':9, 'tail':10}
cat_labels = {'head':1, 'torso':2, 'neck':3, 'lfleg':4, 'lfpa':5, 'rfleg':6, 'rfpa':7, 'lbleg':8, 'lbpa':9, 'rbleg':10, 'rbpa':11, 'tail':12}
cow_labels = {'head':1,'lhorn':2, 'rhorn':3, 'torso':4, 'neck':5, 'lfuleg':6, 'lflleg':7, 'rfuleg':8, 'rflleg':9, 'lbuleg':10, 'lblleg':11, 'rbuleg':12, 'rblleg':13, 'tail':14}
dog_labels = {'head':1,'torso':2, 'neck':3, 'lfleg':4, 'lfpa':5, 'rfleg':6, 'rfpa':7, 'lbleg':8, 'lbpa':9, 'rbleg':10, 'rbpa':11, 'tail':12, 'muzzle':13}
horse_labels = {'head':1,'lfho':2, 'rfho':3, 'torso':4, 'neck':5, 'lfuleg':6, 'lflleg':7, 'rfuleg':8, 'rflleg':9, 'lbuleg':10, 'lblleg':11, 'rbuleg':12, 'rblleg':13, 'tail':14, 'lbho':15, 'rbho':16}
person_labels = {'head':1, 'torso':2, 'neck': 3, 'llarm': 4, 'luarm': 5, 'lhand': 6, 'rlarm':7, 'ruarm':8, 'rhand': 9, 'llleg': 10, 'luleg':11, 'lfoot':12, 'rlleg':13, 'ruleg':14, 'rfoot':15}
sheep_labels = cow_labels
part_labels_combined_parts = {'bird': bird_labels, 'cat': cat_labels, 'cow': cow_labels, 'dog': dog_labels, 'sheep': sheep_labels, 'horse':horse_labels,'person':person_labels}

for k, v in part_labels_combined_parts.items():
    new_map = {}
    for pk, pv in v.items():
        new_map[pv]=pk
    part_labels_combined_parts[k] = new_map
    
for k, v in part_labels_combined_parts.items():
    result.loc[result['obj_class']==k, ['part_labels']] = result.loc[result['obj_class']==k,['part_labels']].replace(v).copy()

result.to_csv(model_path+'/raw_metrics.csv')
res_obj_level = result.groupby(['obj_class', 'id']).mean().reset_index()
res_obj_level.groupby(['obj_class']).mean().reset_index()[['obj_class', 'IOU', 'MSE']].to_csv(model_path+'/obj_level_metrics.csv')
result.groupby(['obj_class', 'part_labels']).mean().reset_index()[['obj_class', 'part_labels',  'IOU', 'MSE']].to_csv(model_path+'/part_level_metrics.csv')

C:\Users\user\Documents\Workspace\MeronymNet-PyTorch\src\evaluation\metrics.py:21: RuntimeWarning: invalid value encountered in true_divide
  iouk = interArea / unionArea


In [30]:
model_path = ('D:/meronym_data/model/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
summary_path = ('D:/meronym_data/runs/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')

result = pd.read_csv(model_path+'/raw_metrics.csv')
result.groupby(['obj_class', 'part_labels']).mean().reset_index()[['obj_class', 'part_labels',  'IOU', 'MSE']]

,obj_class,part_labels,IOU,MSE
0,bird,head,0.573799,0.000412
1,bird,lfoot,0.231832,0.000579
2,bird,lleg,0.340474,0.000492
3,bird,lwing,0.437507,0.001576
4,bird,neck,0.461842,0.000660
...,...,...,...,...
81,sheep,lhorn,0.300649,0.000665
82,sheep,neck,0.634345,0.000288
83,sheep,rhorn,0.215563,0.000987
84,sheep,tail,0.302638,0.000807


In [31]:
res_obj_level = result.groupby(['obj_class', 'id']).mean().reset_index()
res_obj_level.groupby(['obj_class']).mean().reset_index()[['obj_class', 'IOU', 'MSE']]

,obj_class,IOU,MSE
0,bird,0.532596,0.000548
1,cat,0.589477,0.001781
2,cow,0.510379,0.000598
3,dog,0.474684,0.001083
4,horse,0.422377,0.000963
5,person,0.505309,0.001045
6,sheep,0.667891,0.000341


In [32]:
res_obj_level[['IOU', 'MSE']].mean()

IOU    0.525287
MSE    0.000973
dtype: float64

In [ ]:
result.groupby(['obj_class', 'part_labels']).mean().reset_index()[['obj_class', 'part_labels',  'IOU', 'MSE']]

In [ ]:
outfile = 'D:/meronym_data/generate_boxes.npy'
with open(outfile, 'wb') as pickle_file:
    pred_boxes = np.concatenate(pred_boxes)
    pickle.dump(pred_boxes, pickle_file)
outfile = 'D:/meronym_data/generate_boxesobj_class.npy'
with open(outfile, 'wb') as pickle_file:
    pickle.dump(classes,pickle_file)